In [2]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
#from sklearn.model_selection import train_test_split
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

#device = torch.device("cuda:0")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
class SOFDataset(Dataset):
  
  def __init__(self, csv_file):
    # 일부 값중에 NaN이 있음...
    self.dataset = pd.read_csv(csv_file).dropna(axis=0) 
    
    # 중복제거
    self.dataset.drop_duplicates(subset=['Body'], inplace=True)
    
    # Y 값 숫자로 레이블링
    
    self.dataset['Y'] = self.dataset['Y'].map({'LQ_CLOSE':0, 
                                      'LQ_EDIT':1,
                                      'HQ':2})
    
    self.tokenizer = AutoTokenizer.from_pretrained('google/electra-small-discriminator')

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx,[2,5]].values # 데이터셋 따라 idx 변경
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=512,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [4]:
train_dataset = SOFDataset("/home/harock96/hw/project/data/train.csv")
valid_dataset = SOFDataset("/home/harock96/hw/project/data/valid.csv")

                 Id             Y
count  4.500000e+04  45000.000000
mean   4.575616e+07      1.000000
std    7.120035e+06      0.816506
min    3.455266e+07      0.000000
25%    3.973593e+07      0.000000
50%    4.503563e+07      1.000000
75%    5.125584e+07      2.000000
max    6.046802e+07      2.000000
                 Id             Y
count  1.500000e+04  15000.000000
mean   4.576106e+07      1.000000
std    7.139021e+06      0.816524
min    3.455297e+07      0.000000
25%    3.980180e+07      0.000000
50%    4.502755e+07      1.000000
75%    5.130322e+07      2.000000
max    6.047032e+07      2.000000


In [5]:
#model

model = ElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator', num_labels = 3)
model = nn.DataParallel(model)
model.to(device)

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

DataParallel(
  (module): ElectraForSequenceClassification(
    (electra): ElectraModel(
      (embeddings): ElectraEmbeddings(
        (word_embeddings): Embedding(30522, 128, padding_idx=0)
        (position_embeddings): Embedding(512, 128)
        (token_type_embeddings): Embedding(2, 128)
        (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=256, out_features=256, bias=True)
                (key): Linear(in_features=256, out_features=256, bias=True)
                (value): Linear(in_features=256, out_features=256, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output)

In [5]:
#text, attention_mask, y = train_dataset[0]
#model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

In [6]:
# parameter

epochs = 20
batch_size = 32

optimizer = AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

In [9]:
# Learn

losses = []
accuracies = []

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print(i+1, "Train Loss:", total_loss, "Accuracy:", correct.float() / total)

  0%|          | 0/1407 [00:00<?, ?it/s]

Batch Loss: 36.378212720155716 Accuracy: tensor(0.8547, device='cuda:0')
Batch Loss: 69.0741669088602 Accuracy: tensor(0.8619, device='cuda:0')
Batch Loss: 101.04212884604931 Accuracy: tensor(0.8643, device='cuda:0')
Batch Loss: 132.2782952412963 Accuracy: tensor(0.8658, device='cuda:0')
Batch Loss: 162.7401693239808 Accuracy: tensor(0.8668, device='cuda:0')
Batch Loss: 192.42131755501032 Accuracy: tensor(0.8686, device='cuda:0')
Batch Loss: 219.419739253819 Accuracy: tensor(0.8709, device='cuda:0')
Batch Loss: 248.96859212219715 Accuracy: tensor(0.8714, device='cuda:0')
Batch Loss: 276.86037239432335 Accuracy: tensor(0.8727, device='cuda:0')
Batch Loss: 303.42989887297153 Accuracy: tensor(0.8738, device='cuda:0')
Batch Loss: 330.01489901542664 Accuracy: tensor(0.8748, device='cuda:0')
Batch Loss: 357.93020837008953 Accuracy: tensor(0.8751, device='cuda:0')
Batch Loss: 385.09811802208424 Accuracy: tensor(0.8752, device='cuda:0')
Batch Loss: 411.10993602871895 Accuracy: tensor(0.8762, d

  0%|          | 0/1407 [00:00<?, ?it/s]

Batch Loss: 24.903156094253063 Accuracy: tensor(0.8991, device='cuda:0')
Batch Loss: 50.12509021162987 Accuracy: tensor(0.8945, device='cuda:0')
Batch Loss: 74.17527192085981 Accuracy: tensor(0.8957, device='cuda:0')
Batch Loss: 97.76129484921694 Accuracy: tensor(0.8977, device='cuda:0')
Batch Loss: 121.04342135041952 Accuracy: tensor(0.8993, device='cuda:0')
Batch Loss: 143.11332079023123 Accuracy: tensor(0.9012, device='cuda:0')
Batch Loss: 166.36610755324364 Accuracy: tensor(0.9017, device='cuda:0')
Batch Loss: 188.4811251387 Accuracy: tensor(0.9021, device='cuda:0')
Batch Loss: 210.72051333636045 Accuracy: tensor(0.9024, device='cuda:0')
Batch Loss: 233.2149366363883 Accuracy: tensor(0.9027, device='cuda:0')
Batch Loss: 256.47568825259805 Accuracy: tensor(0.9029, device='cuda:0')
Batch Loss: 279.6619789302349 Accuracy: tensor(0.9032, device='cuda:0')
Batch Loss: 300.4759212806821 Accuracy: tensor(0.9043, device='cuda:0')
Batch Loss: 322.1124983392656 Accuracy: tensor(0.9053, device

  0%|          | 0/1407 [00:00<?, ?it/s]

...

Batch Loss: 2.3556867600418627 Accuracy: tensor(0.9906, device='cuda:0')
Batch Loss: 4.623940799443517 Accuracy: tensor(0.9916, device='cuda:0')
Batch Loss: 7.284510224708356 Accuracy: tensor(0.9913, device='cuda:0')
Batch Loss: 9.850865548942238 Accuracy: tensor(0.9912, device='cuda:0')
Batch Loss: 12.381938728445675 Accuracy: tensor(0.9914, device='cuda:0')
Batch Loss: 15.143216425378341 Accuracy: tensor(0.9913, device='cuda:0')
Batch Loss: 18.02858223876683 Accuracy: tensor(0.9912, device='cuda:0')
Batch Loss: 21.538263688096777 Accuracy: tensor(0.9910, device='cuda:0')
Batch Loss: 24.57500802516006 Accuracy: tensor(0.9909, device='cuda:0')
Batch Loss: 27.258251017308794 Accuracy: tensor(0.9908, device='cuda:0')
Batch Loss: 30.045051293389406 Accuracy: tensor(0.9909, device='cuda:0')
Batch Loss: 32.2794152770075 Accuracy: tensor(0.9910, device='cuda:0')
Batch Loss: 34.763291630195454 Accuracy: tensor(0.9910, device='cuda:0')
Batch Loss: 38.54822902486194 Accuracy: tensor(0.9908, dev

  0%|          | 0/1407 [00:00<?, ?it/s]

Batch Loss: 2.192491597379558 Accuracy: tensor(0.9916, device='cuda:0')
Batch Loss: 4.682262751390226 Accuracy: tensor(0.9920, device='cuda:0')
Batch Loss: 6.580956151825376 Accuracy: tensor(0.9921, device='cuda:0')
Batch Loss: 9.088579487171955 Accuracy: tensor(0.9922, device='cuda:0')
Batch Loss: 11.770760982530192 Accuracy: tensor(0.9919, device='cuda:0')
Batch Loss: 14.374441502266563 Accuracy: tensor(0.9919, device='cuda:0')
Batch Loss: 16.813685771019664 Accuracy: tensor(0.9918, device='cuda:0')
Batch Loss: 19.17411237981287 Accuracy: tensor(0.9917, device='cuda:0')
Batch Loss: 22.326697670127032 Accuracy: tensor(0.9915, device='cuda:0')
Batch Loss: 26.273814432119252 Accuracy: tensor(0.9910, device='cuda:0')
Batch Loss: 29.02026687338366 Accuracy: tensor(0.9908, device='cuda:0')
Batch Loss: 31.753997957828688 Accuracy: tensor(0.9909, device='cuda:0')
Batch Loss: 34.715837049501715 Accuracy: tensor(0.9909, device='cuda:0')
Batch Loss: 37.45010257695685 Accuracy: tensor(0.9909, de

  0%|          | 0/1407 [00:00<?, ?it/s]

Batch Loss: 2.3841634393902496 Accuracy: tensor(0.9934, device='cuda:0')
Batch Loss: 4.32438068970805 Accuracy: tensor(0.9931, device='cuda:0')
Batch Loss: 6.7434532135375775 Accuracy: tensor(0.9927, device='cuda:0')
Batch Loss: 8.799369528249372 Accuracy: tensor(0.9929, device='cuda:0')
Batch Loss: 11.009234417113476 Accuracy: tensor(0.9929, device='cuda:0')
Batch Loss: 12.915206524310634 Accuracy: tensor(0.9931, device='cuda:0')
Batch Loss: 15.589087247033603 Accuracy: tensor(0.9926, device='cuda:0')
Batch Loss: 17.93497456255136 Accuracy: tensor(0.9927, device='cuda:0')
Batch Loss: 20.539291825785767 Accuracy: tensor(0.9923, device='cuda:0')
Batch Loss: 23.15426414652029 Accuracy: tensor(0.9924, device='cuda:0')
Batch Loss: 25.767018212587573 Accuracy: tensor(0.9924, device='cuda:0')
Batch Loss: 28.309997025702614 Accuracy: tensor(0.9922, device='cuda:0')
Batch Loss: 30.99888444028329 Accuracy: tensor(0.9921, device='cuda:0')
Batch Loss: 33.47585130634252 Accuracy: tensor(0.9920, de

In [10]:
# 모델 저장하기

torch.save(model.state_dict(), "model_electra_small.pt")

In [11]:
# 모델 불러오기

model.load_state_dict(torch.load("model_electra_small.pt"))

<All keys matched successfully>

In [12]:
# model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(valid_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/469 [00:00<?, ?it/s]

Accuracy: tensor(0.9134, device='cuda:0')
